# PROYECTO ICE3333 - Reconstrucción de la configuracion pulmonar libre de tensiones mediante metodos inversos

Por : Elias Basulto - Carlos Olguin

## Testeos

In [ ]:
# --- abrir_malla_deformada.py ---
# Este script carga la malla original y el campo de desplazamiento final
# desde un archivo ADIOS2 (.bp) y reconstruye el objeto de la malla
# deformada en DOLFINx, dejándolo listo para su uso.

from mpi4py import MPI
import numpy as np
from dolfinx import mesh, fem, io
from pathlib import Path
import adios4dolfinx as adx

# --- Configuración de Archivos ---
# Ruta al archivo de entrada que contiene los resultados de la simulación directa
input_bp = "/home/ebasu/fenics_out/viga.bp"

# Ruta para guardar un archivo de visualización (opcional, pero muy recomendado)
output_dir = Path("/home/ebasu/fenics_out/verificacion_carga")
output_dir.mkdir(parents=True, exist_ok=True)
output_xdmf = str(output_dir / "malla_deformada_cargada.xdmf")

# --- 1. Leer la Malla Base y el Desplazamiento ---
print(f"Cargando datos desde: {input_bp}")

# a) Leemos la malla original (recta) desde el archivo.
domain_original = adx.read_mesh(input_bp, comm=MPI.COMM_WORLD)
print("-> Malla original leída correctamente.")

# b) Creamos el espacio de funciones sobre la malla original y una Función para almacenar el desplazamiento.
V_original = fem.functionspace(domain_original, ("Lagrange", 1, (3,)))
u_desplazamiento = fem.Function(V_original)

# c) Leemos los datos del campo de desplazamiento guardado en el último paso (t=1.0).
adx.read_function(input_bp, u_desplazamiento, name="Displacement", time=1.0)
print("-> Campo de desplazamiento leído correctamente.")

# --- 2. Crear el Objeto de la Malla Deformada ---
print("\nConstruyendo el objeto de la malla deformada en memoria...")

# a) Obtenemos la topología (conectividad) de la malla original.
tdim = domain_original.topology.dim
num_cells_local = domain_original.topology.index_map(tdim).size_local
cell_connectivity = domain_original.topology.connectivity(tdim, 0).array.reshape(num_cells_local, -1)

# b) Calculamos las nuevas coordenadas sumando el desplazamiento.
deformed_coords = domain_original.geometry.x + u_desplazamiento.x.array.reshape(-1, 3)

# c) Creamos el nuevo objeto de malla usando los argumentos correctos.
domain_deformado = mesh.create_mesh(
    MPI.COMM_WORLD, cell_connectivity, deformed_coords, domain_original.ufl_domain()
)

print("\n¡Éxito! El objeto 'domain_deformado' está listo para usarse en Python.")

# --- 3. Verificación (Opcional) ---
# Guardamos una copia de esta malla reconstruida para asegurarnos de que se ve bien en ParaView.
with io.XDMFFile(MPI.COMM_WORLD, output_xdmf, "w") as xdmf:
    xdmf.write_mesh(domain_deformado)

if MPI.COMM_WORLD.rank == 0:
    print(f"\nSe ha guardado una copia de la malla deformada en: {output_xdmf}")

## Testeos Poroeslatico (inverse Poroelasitcity Problem IPP)

Basado en Paper 4.1, problemas en torno a la aceleración y el tensiones residuales pero sirve de esqueleto pensar.

In [ ]:
# La idea de este script es correr lo mismo que el jupyter pero con terminal, evitando incompatibilidades de kernel de jupyter con librerias



from mpi4py import MPI
import numpy as np
import ufl
from dolfinx import mesh, fem, io, nls, log
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
import basix.ufl

# Malla y Espacio de Funciones (2D, memoria) 
domain = mesh.create_unit_cube(MPI.COMM_WORLD, 1, 1, 1) 
gdim = domain.geometry.dim

cell_name_str = domain.topology.cell_name() 
P2_vec = basix.ufl.element("Lagrange", cell_name_str, 2, shape=(gdim,))
P1_scal = basix.ufl.element("Lagrange", cell_name_str, 1)

W_elem = basix.ufl.mixed_element([P2_vec, P1_scal, P1_scal, P1_scal])
W = fem.functionspace(domain, W_elem)

# Variables y Funciones 
wh = fem.Function(W)
wh_o = fem.Function(W) 

(d_star, phi0_star, lam_star, mu_star) = ufl.TestFunctions(W)

d, phi0, lam, mu = ufl.split(wh)
d_o, phi0_o, lam_o, mu_o = ufl.split(wh_o)

# Parámetros (Basados paper)
dt_val = 0.01                 # Paso de tiempo (dt) 
t_ramp = 0.1                  # Tiempo de rampa de carga 
t_final = 2.0                 # Tiempo final
num_steps = int(t_final / dt_val)
dt = fem.Constant(domain, dt_val)

# Poroelasticidad
k_val = 2.0e-7                
rho_f = fem.Constant(domain, 1.0) 
phi_bar = fem.Constant(domain, 0.1) 

# Carga (Fuente)
pa = fem.Constant(domain, 10000.0)   
beta_max = 1.0e-4                     
beta = fem.Constant(domain, 0.0)      

# Cinematica y tensores

I = ufl.Identity(gdim)
f = I + ufl.grad(d) #
j = ufl.det(f)      
F = ufl.inv(f)      
J = ufl.det(F)     

# Modelo material hipereslastico

C = fem.Constant(domain, 880.0) 
B = fem.Constant(domain, 5.0e4)

# Anisotropia parametros inspo script paper

bff = 20.0
bss = 10.0
bnn = 20.0
bfs = 10.0
bfn = 10.0
bsn = 10.0

# Direcciones de fibra (alineadas con los ejes para el cubo) 
f0 = ufl.as_vector([1.0, 0.0, 0.0])
s0 = ufl.as_vector([0.0, 1.0, 0.0])
n0 = ufl.as_vector([0.0, 0.0, 1.0])

C_tensor = F.T * F
E_green = 0.5 * (C_tensor - I) # Tensor de Green-Lagrange

E_ff = ufl.inner(E_green * f0, f0)
E_ss = ufl.inner(E_green * s0, s0)
E_nn = ufl.inner(E_green * n0, n0)
E_fs = ufl.inner(E_green * f0, s0)
E_fn = ufl.inner(E_green * f0, n0)
E_sn = ufl.inner(E_green * s0, n0)

# Exponente Q 
Q = bff * E_ff**2 + bss * E_ss**2 + bnn * E_nn**2 + \
    bfs * (E_fs**2) + bfn * (E_fn**2) + bsn * (E_sn**2)

# Anisotropica
S_ff = 2.0 * bff * E_ff
S_ss = 2.0 * bss * E_ss
S_nn = 2.0 * bnn * E_nn
S_fs = 2.0 * bfs * E_fs
S_fn = 2.0 * bfn * E_fn
S_sn = 2.0 * bsn * E_sn

S = S_ff * ufl.outer(f0, f0) + S_ss * ufl.outer(s0, s0) + S_nn * ufl.outer(n0, n0) + \
    S_fs * (ufl.outer(f0, s0) + ufl.outer(s0, f0)) + \
    S_fn * (ufl.outer(f0, n0) + ufl.outer(n0, f0)) + \
    S_sn * (ufl.outer(s0, n0) + ufl.outer(n0, s0))

P_aniso = C * ufl.exp(Q) * (F * S)

# Volumetrica

P_vol = B * J**2 * ufl.inv(F.T)

P = P_aniso + P_vol

# Parametros presion paper script basado

q1 = fem.Constant(domain, 1.333)
q2 = fem.Constant(domain, 550.0)
q3 = fem.Constant(domain, 10.0)

def dp_P(Phi):
    # Agregamos 'eps' para estabilidad numérica, evitando ln(0)
    eps = fem.Constant(domain, 1e-10)
    return (q1/q3) * ufl.exp(q3 * Phi) + q2 * ufl.ln(q3 * Phi + eps)

Phi_actual = J * phi_bar # Phi = J * phi_bar
Phi_ref = phi0

p_tilde = dp_P(Phi_actual) - dp_P(ufl.conditional(ufl.gt(Phi_ref, 0.0), Phi_ref, 1e-10))

theta = -beta * (mu - pa) # mu es la variable de presión mixta 


phi0_dt = (phi0 - phi0_o) / dt

# Equilibrio Mecánico 
F_mech = (ufl.inner(j * P * ufl.inv(f.T), ufl.grad(d_star)) + ufl.inner(lam, ufl.div(d_star))) * ufl.dx

# Incompresibilidad
F_incomp = ufl.inner(j * (1 - phi0) - (1 - phi_bar), lam_star) * ufl.dx

# Ecuación de Flujo (con k_val y theta)
F_poro_A = (-ufl.inner(phi0_dt, mu_star) \
            + ufl.inner(k_val * ufl.grad(mu), ufl.grad(mu_star)) \
            - ufl.inner(1/rho_f * theta, mu_star)) * ufl.dx

# Definición de Presión
F_poro_B = ufl.inner(mu - p_tilde, phi0_star) * ufl.dx

F_total = F_mech + F_incomp + F_poro_A + F_poro_B

# Condiciones Nulas testeo rapido

def left_boundary(x):
    return np.isclose(x[0], 0)
left_facets = mesh.locate_entities_boundary(domain, gdim - 1, left_boundary)
left_dofs_x = fem.locate_dofs_topological(W.sub(0).sub(0), gdim - 1, left_facets)
bc_left_x = fem.dirichletbc(0.0, left_dofs_x, W.sub(0).sub(0))

def bottom_boundary(x):
    return np.isclose(x[1], 0)
bottom_facets = mesh.locate_entities_boundary(domain, gdim - 1, bottom_boundary)
bottom_dofs_y = fem.locate_dofs_topological(W.sub(0).sub(1), gdim - 1, bottom_facets)
bc_bottom_y = fem.dirichletbc(0.0, bottom_dofs_y, W.sub(0).sub(1))

def front_boundary(x):
    return np.isclose(x[2], 0)

front_facets = mesh.locate_entities_boundary(domain, gdim - 1, front_boundary)
front_dofs_z = fem.locate_dofs_topological(W.sub(0).sub(2), gdim - 1, front_facets)
bc_front_z = fem.dirichletbc(0.0, front_dofs_z, W.sub(0).sub(2))

bcs = [bc_left_x, bc_bottom_y, bc_front_z] 

# Solver no lineal
problem = NonlinearProblem(F_total, wh, bcs=bcs)
solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-6 
solver.max_it = 50 

log.set_log_level(log.LogLevel.INFO)

phi0_init_expr = fem.Expression(phi_bar, W.sub(1).element.interpolation_points())
wh_o.sub(1).interpolate(phi0_init_expr)
wh.sub(1).interpolate(phi0_init_expr)

print(f"Iniciando simulación IPP (Test 2 - Cubo 3D) [fuente: 432]...")
print(f"Parámetros: dt={dt_val}, t_ramp={t_ramp}, k={k_val}, pa={pa.value}")

V_out_P1_vec = fem.functionspace(domain, ("Lagrange", 1, (gdim,)))
d_out = fem.Function(V_out_P1_vec)
d_out.name = "Desplazamiento_Inverso_P1"

phi0_sol = wh.sub(1)
mu_sol = wh.sub(3)
phi0_sol.name = "Porosidad_Referencia"
mu_sol.name = "Presion_Mixta"

xdmf = io.XDMFFile(MPI.COMM_WORLD, "IPP_cubo_resultado.xdmf", "w")
xdmf.write_mesh(domain)

d_hat_sol_P2 = wh.sub(0) 
d_out.interpolate(d_hat_sol_P2) 

xdmf.write_function(d_out, 0.0) 
xdmf.write_function(phi0_sol, 0.0)
xdmf.write_function(mu_sol, 0.0)

norm_phi0_form = fem.form(ufl.inner(phi0_sol, phi0_sol) * ufl.dx)
norm_phi0_o_form = fem.form(ufl.inner(wh_o.sub(1), wh_o.sub(1)) * ufl.dx)

for i in range(num_steps):
    t_current = (i + 1) * dt_val
    
    ramp_factor = min(1.0, t_current / t_ramp)
    beta.value = beta_max * ramp_factor
    
    print(f"Paso {i+1}/{num_steps}, (Pseudo) Tiempo {t_current:.3f}, Carga (beta): {ramp_factor*100:.1f}%")

    try:
        n_iter, converged = solver.solve(wh)
        if not converged:
            print("Newton no convergió.")
            break
    except Exception as e:
        print(f"Fallo el solver: {e}")
        break

    wh_o.x.array[:] = wh.x.array
    
    if (i+1) % 10 == 0:
        d_out.interpolate(wh.sub(0)) 
        xdmf.write_function(d_out, t_current) 
        xdmf.write_function(phi0_sol, t_current)
        xdmf.write_function(mu_sol, t_current)
    
    norm_val = np.sqrt(fem.assemble_scalar(norm_phi0_form))
    norm_val_o = np.sqrt(fem.assemble_scalar(norm_phi0_o_form))
    norm_phi0_change = norm_val - norm_val_o
    
    if np.abs(norm_phi0_change) < 1e-7 and ramp_factor == 1.0:
        print(f"Convergencia de estado estacionario alcanzada en el paso {i+1}.")
        break

xdmf.close()
print("Simulación terminada.")

norm_val_final = np.sqrt(fem.assemble_scalar(norm_phi0_form))
print(f"Norma L2 de la porosidad de referencia (phi0) final: {norm_val_final:.4e}")

## Testeos Poroeslatico (DPP)

In [1]:
# MALO:

# La idea de este script es correr lo mismo que el jupyter pero con terminal, evitando incompatibilidades de kernel de jupyter con librerias

from mpi4py import MPI
import numpy as np
import ufl
from dolfinx import mesh, fem, io, nls, log
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
import basix.ufl


domain = mesh.create_unit_cube(MPI.COMM_WORLD, 5,5,5) 
gdim = domain.geometry.dim

cell_name_str = domain.topology.cell_name() 
P2_vec = basix.ufl.element("Lagrange", cell_name_str, 2, shape=(gdim,))
P1_scal = basix.ufl.element("Lagrange", cell_name_str, 1)

# Estructura Final DPP: [u, p] (Desplazamiento, Presión)
W_elem = basix.ufl.mixed_element([P2_vec, P1_scal])
W = fem.functionspace(domain, W_elem)

# Variables
wh = fem.Function(W)
wh_o = fem.Function(W) 
(v, q) = ufl.TestFunctions(W) 

u, p = ufl.split(wh)
u_o, p_o = ufl.split(wh_o)

dt_val = 0.001 
t_ramp = 0.5 
t_final = 1.0 
num_steps = int(t_final / dt_val)
dt = fem.Constant(domain, dt_val)

k_perm = fem.Constant(domain, 2.0e-7) 
rho_f = fem.Constant(domain, 1000.0)
rho_solid = fem.Constant(domain, 2000.0) 
f_body = rho_solid * ufl.as_vector([0.0, 0.0, -9.8]) # Gravedad

# Material Mecánico (Usyk/Bruinsma)
C = fem.Constant(domain, 880.0); B = fem.Constant(domain, 5.0e4);
M_biot = fem.Constant(domain, 1e9) # Módulo de Biot

# Carga de Presión Pleural (Tracción Neumann)
pleural_pressure_max = 500.0 
pleural_pressure = fem.Constant(domain, 0.0) 

# --- Usyk Anisotropía ---
bff = 20.0; bss = 10.0; bnn = 20.0; bfs = 10.0; bfn = 10.0; bsn = 10.0;
f0 = ufl.as_vector([1.0, 0.0, 0.0]); s0 = ufl.as_vector([0.0, 1.0, 0.0]); n0 = ufl.as_vector([0.0, 0.0, 1.0]);

# --- Cinemática ---
I = ufl.Identity(gdim)
F = I + ufl.grad(u) 
J = ufl.det(F) 
F_inv_T = ufl.inv(F.T)

# --- Usyk P_solid (Primer Piola-Kirchhoff) ---
C_tensor = F.T * F
E_green = 0.5 * (C_tensor - I) 
E_ff = ufl.inner(E_green * f0, f0); E_ss = ufl.inner(E_green * s0, s0); E_nn = ufl.inner(E_green * n0, n0);
E_fs = ufl.inner(E_green * f0, s0); E_fn = ufl.inner(E_green * f0, n0); E_sn = ufl.inner(E_green * s0, n0);
Q = bff * E_ff**2 + bss * E_ss**2 + bnn * E_nn**2 + bfs * (E_fs**2) + bfn * (E_fn**2) + bsn * (E_sn**2)

S_ff = 2.0 * bff * E_ff; S_ss = 2.0 * bss * E_ss; S_nn = 2.0 * bnn * E_nn;
S = S_ff * ufl.outer(f0, f0) + S_ss * ufl.outer(s0, s0) + S_nn * ufl.outer(n0, n0) + ufl.outer(f0, s0) # Simplificado

P_aniso = C * ufl.exp(Q) * (F * S)
P_vol = B * J**2 * F_inv_T
P_solid = P_aniso + P_vol
P_total = P_solid - p * J * F_inv_T # Acoplamiento de Biot (alpha=1)

# --- 3. Almacenamiento de Masa (M_dot) ---
m_storage = (J - 1) + (p / M_biot)
m_storage_o = (ufl.det(I + ufl.grad(u_o)) - 1) + (p_o / M_biot)
m_dot = (m_storage - m_storage_o) / dt

# =============================================================================
# 4. FORMULACIÓN VARIACIONAL (DPP)
# =============================================================================

# Ecuación 1: Equilibrio Mecánico (Momentum)
Res_mech = ufl.inner(P_total, ufl.grad(v)) * ufl.dx - ufl.inner(f_body, v) * ufl.dx 
n_vector = ufl.FacetNormal(domain)
ds = ufl.Measure("ds", domain=domain)
Res_mech -= ufl.inner(pleural_pressure * n_vector, v) * ds # Carga Pleural Neumann

# Ecuación 2: Conservación de Masa (Flujo)
Res_flow = ufl.inner(m_dot, q) * ufl.dx + ufl.inner(k_perm * ufl.grad(p), ufl.grad(q)) * ufl.dx

F_total = Res_mech + Res_flow 

# =============================================================================
# 5. CONDICIONES DE BORDE Y SOLVER
# =============================================================================

# --- BC 1: Fijación (u=0) en un punto (Centro de la Base) ---
V_u, _ = W.sub(0).collapse() 

def bottom_center_point(x):
    return np.logical_and(np.isclose(x[2], 0), 
                          np.logical_and(np.isclose(x[0], 0.5), np.isclose(x[1], 0.5)))

# Localizar DOFs para BC en el subespacio u usando la tupla (sub_space, collapsed_space)
dofs_u_global = fem.locate_dofs_geometrical((W.sub(0), V_u), bottom_center_point)

# Aplanar la lista de arrays de DOFs a un único ndarray
dofs_u_global_flattened = np.concatenate(dofs_u_global)

# Definición de g=0 como una función (necesario para BC vectorial en subespacio)
u_bc_value = fem.Function(V_u)
u_bc_value.interpolate(lambda x: np.zeros((3, x.shape[1]), dtype=np.float64))

# CORRECCIÓN: Si 'g' es una fem.Function, el argumento del espacio V (W.sub(0)) no se usa.
bc_anchor = fem.dirichletbc(u_bc_value, dofs_u_global_flattened)

# --- BC 2: Presión (p=0) en la Tráquea (Superficie Superior) ---
V_p, _ = W.sub(1).collapse() # Espacio colapsado para la presión
def top_surface(x):
    return np.isclose(x[2], 1.0)
top_facets = mesh.locate_entities_boundary(domain, gdim-1, top_surface)

# Localizar DOFs para BC en el subespacio p
dofs_p_global = fem.locate_dofs_topological((W.sub(1), V_p), domain.topology.dim - 1, top_facets)

# Aplanar la lista de arrays de DOFs a un único ndarray
dofs_p_global_flattened = np.concatenate(dofs_p_global)

# Para BC escalar (p=0), podemos usar el valor 0.0 y especificar el subespacio.
bc_pressure = fem.dirichletbc(0.0, dofs_p_global_flattened, W.sub(1))

bcs = [bc_anchor, bc_pressure]


# --- 6. Solver y Loop ---
problem = NonlinearProblem(F_total, wh, bcs=bcs)
solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-6 
solver.max_it = 50 

log.set_log_level(log.LogLevel.INFO)

wh_o.x.array[:] = wh.x.array # Inicializa u_o=0, p_o=0

# =============================================================================
# 7. BUCLE TEMPORAL Y SALIDA (XDMF)
# =============================================================================

print(f"Iniciando simulación DPP (Forward) Pulmón ...")
xdmf = io.XDMFFile(domain.comm, "DPP_Forward_Pulmon_test.xdmf", "w")
xdmf.write_mesh(domain)

# Funciones colapsadas para output robusto
V_u_out, _ = W.sub(0).collapse()
V_p_out, _ = W.sub(1).collapse()
u_out = fem.Function(V_u_out, name="Displacement")
p_out = fem.Function(V_p_out, name="Pressure")

for i in range(num_steps):
    t_current = (i + 1) * dt_val
    
    # Rampa de Presión Pleural (Inhalación)
    ramp_factor = min(1.0, t_current / t_ramp)
    pleural_pressure.value = -pleural_pressure_max * ramp_factor # Presión NEGATIVA
    
    print(f"Paso {i+1}/{num_steps}, Tiempo {t_current:.3f}, Presión Pleural: {pleural_pressure.value:.1f} Pa")

    try:
        n_iter, converged = solver.solve(wh)
        if not converged:
            print("Newton no convergió.")
            break
    except Exception as e:
        print(f"Fallo el solver: {e}")
        break

    wh_o.x.array[:] = wh.x.array 
    
    # Guardar resultados (usando funciones colapsadas)
    wh.sub(0).copy(result=u_out)
    wh.sub(1).copy(result=p_out)
    
    # Guardar cada 5 pasos para aligerar la salida
    if (i+1) % 5 == 0:
        xdmf.write_function(u_out, t_current)
        xdmf.write_function(p_out, t_current)

xdmf.close()
print("Simulación DPP Terminada. Resultados en 'DPP_Forward_Pulmon.xdmf'")

c:\Users\carlo\AppData\Local\Programs\Python\Python312\Lib\site-packages\ufl\__init__.py:250: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


ModuleNotFoundError: No module named 'dolfinx'

## Firedrake